<a href="https://colab.research.google.com/github/Lucas-C-Vargas/Image-Transmission-via-Audio-File/blob/main/Image_to_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Final - Transmissão de Imagens via Arquivos de Áudio
---

*IMAGEM → ÁUDIO*

**Alunos:**
- Lucas de Carvalho Vargas (0039109)
- Pedro Henrique Lopes Matias (0039112)

### Bibliotecas

In [ ]:
import numpy as np # Numpy para produção de matrizes
import matplotlib.pyplot as plt # Matplotlib para plotar imagens
import scipy as sp
from scipy import signal
from scipy.io import wavfile # Usado para processar arquivos .WAV
from scipy.fft import fft
import requests # Obter arquivos em repositórios do GitHub
from PIL import Image # Usado para processar imagens
from IPython.display import Audio # Usado para processar arquivos de audio
import os.path
from os import path # Navega nos diretórios

import cv2 as cv # Open CV usado para abertura de imagens
import urllib.request as url  # urllib para fazer download de imagens em repositório do github
import zipfile # zipfile para extração de arquivos zip
from math import sqrt, cos, sin, pi

In [ ]:
# backup (referencias)
from scipy import signal
import cv2 as cv # Open CV usado para abertura de imagens
import urllib.request as url  # urllib para fazer download de imagens em repositório do github
import zipfile # zipfile para extração de arquivos zip em
from math import sqrt, cos, sin, pi

### Funções

In [ ]:
def grayscale(rgb=[0, 0, 0]):
    '''
    Calculates and returns the Grayscale component for a RGB color.
    '''

    color = int(0.30*rgb[0] + 0.59*rgb[1] + 0.11*rgb[2]) # Escala de cinza ponderada
    return color

In [ ]:
def fourier(x, t, Hz=True, dB=False, deg=False, Nfft=None):
    '''
    Calculates the Magnitude, Phase e Frequencies of a signal x(t).
    ```

    # PARAMETERS:
    - x: array_like with amplitudes (time domain)
    - t: array_like with time stamps (time domain)
    - Hz: boolean, optional
       >>> False → rad/s
       >>> True → Hz
    - dB: boolean, optional
       >>> False → no units
       >>> True → dB
    - deg: boolean, optional
       >>> False → rad
       >>> True → degrees
    - Nfft: int, optional

    # RETURNS:
    - mag: array_like
    - phase: array_like
    - freq: array_like
    ```
    '''

    N = len(x) # Quantidade de amostras do vetor x

    # Se Nfft não for declarado, utiliza os N pontos do sinal
    if Nfft==None: Nfft=N

    T = np.mean(t[1:] - t[:-1]) # Período de amostragem

    # Calcula FFT
    Xf = fft(x, n=Nfft)/N # Corrige a amplitude na frequencia
    Xf[1:] = 2*Xf[1:] # Dobra amplitudes
    Xf = Xf[:Nfft//2] # Descarta simetria na frequencia

    # Obtém módulo e fase da FFT
    mag = np.abs(Xf)
    phase = np.angle(Xf, deg)

    # Verifica cálculo da amplitude em dB
    if dB:
        mag = 20*np.log10(mag) # Converte amplitude em dB

    # Calcula o vetor de frequências
    if Hz:
        # Vetor de frequências [Hz]
        freq0 = 1/(T*Nfft)
        freq = np.arange(0, Nfft*freq0, freq0)
    else:
        # Vetor de frequências [rad/s]
        w0 = 2*np.pi/(T*Nfft)
        freq = np.arange(0, Nfft*w0, w0)
    freq = freq[:Nfft//2] # Descarta simetria na frequencia

    return mag, phase, freq

In [ ]:
def extract_pixels(image, RGB=False):
    '''
    Returns an array with the color components for every pixel, added by 1.
    Starts on the upper left pixel, moving horizontally, on a zigzag patern.
    The signal is inverted on every "line" break.
    ```

    # PARAMETERS:
    - image: PIL.Image.Image
    - RGB: boolean, optional
    >>> False → Grayscale (1 value per pixel)
    >>> True → RGB (3 values per pixel)

    # RETURNS:
    - signal: array_like
    ```
    '''

    width, height = image.size # Dimensões da imagem
    signal = [] # Inicia um vetor vazio

    if RGB: # Sinal com 3 componentes por pixel (RGB)
        for y in range(height):
            if y%2 == 0: # Insere as informações das componentes RGB no vetor, com sinal positivo
                for x in range(width):
                    rgb = image.getpixel((x, y)) # Componentes RGB do pixel
                    signal.append(rgb[0]+1) # Red
                    signal.append(rgb[1]+1) # Green
                    signal.append(rgb[2]+1) # Blue
            else: # Insere as informações das componentes RGB no vetor, com sinal negativo
                for x in range(width-1, -1, -1):
                    rgb = image.getpixel((x, y)) # Componentes RGB do pixel
                    signal.append(-rgb[0]-1) # Red
                    signal.append(-rgb[1]-1) # Green
                    signal.append(-rgb[2]-1) # Blue
    else: # Sinal com 1 componente por pixel (Grayscale)
        for y in range(height):
            if y%2 == 0: # Insere a informação da componente Grayscale no vetor, com sinal positivo
                for x in range(width):
                    rgb = image.getpixel((x, y)) # Componentes RGB do pixel
                    color = grayscale(rgb) # Converte para escala de cinza (ponderada)
                    signal.append(color+1)
            else: # Insere a informação da componente Grayscale no vetor, com sinal negativo
                for x in range(width-1, -1, -1):
                    rgb = image.getpixel((x, y)) # Componentes RGB do pixel
                    color = grayscale(rgb) # Converte para escala de cinza (ponderada)
                    signal.append(-color-1)

    return signal

In [ ]:
def calc_dimensions(signal, RGB=False):
    '''
    Calculates and returns the dimensions from the original image.
    ```

    # PARAMETERS:
    - signal: array_like
    - RGB: boolean, optional
    >>> False → Grayscale (1 value per pixel)
    >>> True → RGB (3 values per pixel)

    # RETURNS:
    - width: int (pixels)
    - heigth: int (pixels)
    ```
    '''

    height = 1; # Inicia o contador de linhas

    # Detecta estado inicial de sinal das componentes
    if signal[0] >= 0:
        positive = True
    else:
        positive = False

    # Detecta mudança de sinal das componentes (quebra de linha)
    for x in range(len(signal)):
        if positive and (signal[x] < 0): # Borda de descida
            positive = False
            height = height + 1
        if not(positive) and (signal[x] > 0): # Borda de subida
            positive = True
            height = height + 1

    # Calcula largura da imagem de acordo com o sistema de cores utilizado
    width = len(signal)/height
    if RGB:
        width = width/3

    return int(width), int(height)

### Entrada (Imagem)

In [ ]:
# Importa o drive
# from google.colab import drive
# drive.mount('/content/drive');

# Clona o repositório do projeto
!git clone https://github.com/Lucas-C-Vargas/Image-Transmission-via-Audio-File.git Git

fatal: destination path 'Git' already exists and is not an empty directory.


In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
image = Image.open(fn)
width, heigth = image.size
image

### Conversão da imagem em audio

In [ ]:
extrgb = True
sigext = extract_pixels(image,extrgb)
fs = 44100

tempo = np.arange(0, len(sigext)) / fs

modulacao = True

frequencia_portadora = 12345
sigport = np.cos(2*np.pi*frequencia_portadora * tempo)
if modulacao:
  modsig = sigport * sigext
else:
  modsig = sigext

plt.figure(figsize=(12,2))
plt.plot(tempo, modsig, color = "#2B2B2B")

if extrgb:
  plt.title("Imagem convertida - RGB")
  plt.ylabel("Cor em RGB")
else:
  plt.title("Imagem convertida - Escala de cinza")
  plt.ylabel("Cor em escala de cinza")

plt.xlabel("Tempo(s)");
# plt.xlim(0, 6000)

magsi, phasi, freqsi = fourier(modsig, tempo)

plt.figure(figsize=(12,2))
plt.plot(freqsi, magsi, color = "#2B2B2B")

if extrgb:
  plt.title("FFT - Imagem convertida - RGB")
  plt.ylabel("Amplitude")
else:
  plt.title("FFT - Imagem convertida - Escala de cinza")
  plt.ylabel("Amplitude")

plt.xlabel("Frequência(Hz)");
# plt.xlim(7000, 16000)
Audio(modsig, rate=fs, normalize=True)


In [ ]:
rinf = np.zeros(60)
ginf = np.zeros(60)
binf = np.zeros(60)

for xp in range(0, 30-1, 3):
  rinf[xp] = rinf[xp] + 8
  ginf[xp+1] = ginf[xp+1] + 6
  binf[xp+2] = binf[xp+2] + 13

for xn in range(30, 60-1, 3):
  rinf[xn] = rinf[xn] - 8
  ginf[xn+1] = ginf[xn+1] - 6
  binf[xn+2] = binf[xn+2] - 13

xinf = np.arange(60)

plt.figure(figsize=(10,3))
plt.bar(xinf, rinf, color = "Red", label = "Red")
plt.bar(xinf, ginf, color = "Green", label = "Green")
plt.bar(xinf, binf, color = "Blue", label = "Blue")
plt.title("Representação da imagem em sinal")
plt.ylabel("Quantidade de cor / Pixel")
plt.xlabel("RGB por pixel")
plt.legend();


In [ ]:
# Image Transmission via Audio File

# Image to Audio
'''

'''

# Audio to Image
'''
1 - Inserir arquivo de audio
2 - Obter sinal "criptografado" (demodulação e/ou demultiplexação)
3 - Reconstruir imagem
'''

[//]: Rodapé
# <center> --- </center>

---

<center>
  <img
    src="https://drive.google.com/uc?export=view&id=1nHTWpJNRG-PMsewxoHzlCYARkTwPQIrw"
    alt="Empty_PNG"
    align="left"
    height="50"
    width="50"
  />
  <img
    src="https://drive.google.com/uc?export=view&id=1m_rzpQJQOZ1Kd3RBA5KkY0dm-_Swlg4_"
    alt="Logo IFMG"
    align="left"
    height="50"
  />
  <img
    src="https://drive.google.com/uc?export=view&id=1nHTWpJNRG-PMsewxoHzlCYARkTwPQIrw"
    alt="Empty_PNG"
    align="right"
    height="50"
    width="50"
  />
  <img
    src="https://drive.google.com/uc?export=view&id=1m_rzpQJQOZ1Kd3RBA5KkY0dm-_Swlg4_"
    alt="Empty_PNG"
    align="right"
    height="50"
    width="50"
  />
  <figcaption>Instituto Federal de Educação, Ciência e tecnologia de Minas Gerais - Campus Betim</figcaption>
  <figcaption>Processamento de Sinais - 2023.2</figcaption>
</center>

---